## Github Actions and ACT


### Terminology

- job      -> a shell script or action
- action   -> a custom application usually on the Github Marketplace
- workflow -> one or more jobs
- runner   -> [ubuntu-lastest,macos-latest,windows-latest]
- events   -> [push, PR, manual trigger]

https://github.com/actions/runner-images/blob/main/images/linux/Ubuntu2004-Readme.md

### Triggers

```yaml
on:
  push:
    branches:
    - main
    - 'releases/**'
  pull_request:
  workflow_dispatch:
  schedule:
    - cron:  '0 22 1 * *'

```

### Variables

```yaml
env:
  TAG_NAME: "tag_${{ github.sha }}_${{ github.run_attempt }}"
```

```yaml
run: echo ${{ github.repository }} ${{ env.TAG_NAME }}
```

### Combining Them

```yaml
name: Example 1 Workflow
on:
  push:
  workflow_dispatch:
env:
  TAG_NAME: "tag/${{ github.sha }}/${{ github.run_attempt }}"
jobs:
  example1:
    name: Example 1 Job
    runs-on: ubuntu-latest
    steps:
    - name: Example 1 Action
      run: echo ${{ github.repository }} ${{ env.TAG_NAME }}
```

### Running Example1

In [20]:
!gh workflow run example1.yaml --ref main

✓ Created workflow_dispatch event for example1.yaml at main

To see runs for this workflow, try: gh run list --workflow=example1.yaml


In [22]:
!gh run list --workflow example1.yaml --limit 1

STATUS  NAME          WORKFLOW     BRANCH  EVENT        ID          ELAPSED  AGE
✓       changed p...  Example ...  main    workflow...  2848542299  12s      1m


### Message Passing


```yaml
- name: Format Tag Name String
  id: format_tag_name
  run: |
    tag_name="${{ env.TAG_NAME }}"
    formatted_tag_name=${tag_name//\//-} # replace forward slash with -
    echo "::set-output name=FORMATTED_TAG_NAME::$formatted_tag_name"
```

```yaml
- name: Echo Formatted Tag Name
  run: echo ${{ steps.format_tag_name.outputs.FORMATTED_TAG_NAME }}
```

### Running Example2

In [ ]:
!gh workflow run example2.yaml --ref main

In [ ]:
!gh run list --workflow example2.yaml --limit 1

### External Workflows

```yaml
jobs:
  example4:
    name: Example 3 Job
    runs-on: ubuntu-latest
    env:
      TAG_NAME: "release/${{ github.sha }}/${{ github.run_attempt }}"
    outputs:
      branch_type: ${{ steps.release_branch_check.outputs.branch_type }}
      tag_name: ${{ env.TAG_NAME }}
    steps:
    - name: Check if branch is release
      id: release_branch_check
      run: |
        pattern="(^release\/.*|^hotfix\/.*)"
        if [[ ${{ env.TAG_NAME }} =~ $pattern ]]
        then
          echo "::set-output name=branch_type::release"
        fi   
```

```yaml
example3_2:
if: needs.example3.outputs.branch_type == 'release'
    needs: example3
    uses: evan-certn/actions-ttt/.github/workflows/example.yaml@main
    with:
        TAG_NAME: needs.example3.outputs.tag_name
```

```yaml
name: Echo Tag from reusable workflow
on:
  workflow_call:
    inputs:
      TAG_NAME:
        type: string
        required: true
jobs:
  Example3_3:
    runs-on: ubuntu-latest
    steps:
    - name: Echo Tag Name
      run: echo ${{ inputs.TAG_NAME }}
```

In [24]:
!gh run list --workflow example3.yaml --limit 1

STATUS  NAME              WORKFLOW       BRANCH  EVENT  ID          ELAPSED  AGE
✓       fixed example...  Example 3 ...  main    push   2848791015  23s      0m


## ACT

In [32]:
!act \
    --container-architecture linux/amd64 \
    -P ubuntu-latest=ubuntu:latest \
    -W .github/workflows/example5.yaml

[Example 5 Workflow/Example 5 Job] 🚀  Start image=ubuntu:latest
[Example 5 Workflow/Example 5 Job]   🐳  docker pull image=ubuntu:latest platform=linux/amd64 username= forcePull=false
[Example 5 Workflow/Example 5 Job]   🐳  docker create image=ubuntu:latest platform=linux/amd64 entrypoint=["/usr/bin/tail" "-f" "/dev/null"] cmd=[]
[Example 5 Workflow/Example 5 Job]   🐳  docker run image=ubuntu:latest platform=linux/amd64 entrypoint=["/usr/bin/tail" "-f" "/dev/null"] cmd=[]
[Example 5 Workflow/Example 5 Job]   🐳  docker exec cmd=[mkdir -m 0777 -p /var/run/act] user=root workdir=
[Example 5 Workflow/Example 5 Job] ⭐  Run Example 5 Action with ACT
[Example 5 Workflow/Example 5 Job]   🐳  docker exec cmd=[bash --noprofile --norc -e -o pipefail /var/run/act/workflow/1] user= workdir=
| evan-certn/actions-ttt run with ACT
[Example 5 Workflow/Example 5 Job]   ✅  Success - Example 5 Action with ACT


### Problems with Act
- No access to certain github enviroment variables
- docker inside docker requies setup
- images are not 1:1